# 📚 Kapitola 33: LangChain základy

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 4 | EXPERT</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **LangChain installation**
- **Core concepts: Chains, Prompts**
- **Memory types**
- **Document loaders variety**
- **Text splitters strategies**
- **Output parsers**
- **LLM wrappers**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 32

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 33

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 33: LangChain základy")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# LangChain základy

## 1. Úvod a motivace

LangChain je jedna z nejvýznamnějších knihoven v oblasti **generativního AI**, která umožňuje efektivní tvorbu, ladění a nasazování aplikací založených na jazykových modelech (LLMs). Pokud se zajímáš o vývoj AI aplikací, které pracují s textem, vytvářejí obsah nebo interagují s uživateli, pak LangChain je nástroj, který ti umožní překonat hranice jednotlivých LLM modulů a vytvářet komplexní, funkční řešení.

V praxi se LangChain používá například v chatbotech, systémech podpory klientů, automatizovaném generování obsahu, analýze dokumentů nebo v oblastech jako je vzdělávání a právní poradenství. Základní myšlenka je jednoduchá: **vytvořit řetězce (chains) komponent, které pracují spolu, aby zvládly složitější úkoly** – například zpracování dotazu, vyhledání informací, a následné generování odpovědi.

Po absolvování této kapitoly budeš schopen:
- Nainstalovat a nastavit LangChain v prostředí Pythonu
- Vytvářet jednoduché řetězce (chains) mezi jednotlivými komponenty
- Pochopit základní koncepty, jako jsou prompts, memory, output parsers a další
- Získat přehled o tom, jak přizpůsobit LLM podle potřeby

---

## 2. Hlavní koncepty

### LangChain instalace (LangChain installation)

Nejdříve je nutné nainstalovat LangChain v Pythonu – obvykle prostřednictvím `pip`:

```bash
pip install langchain
```

V závislosti na používaném LLM (např. OpenAI, Hugging Face) můžeš také instalovat dodatečné balíčky:

```bash
pip install openai langchain-community
```

Při instalaci si všimni, že LangChain je modulární – podporuje různé LLM, databáze, prompty a další komponenty. Tato modularity umožňuje velkou flexibilitu při tvorbě aplikací.

---

### Řetězce (Chains) a Prompty (Prompts)

**Řetězec (Chain)** je způsob, jak spojit několik komponent dohromady. Například: získání vstupu → předzpracování → zadání do LLM → analýza výstupu.

Jeden z nejčastějších typů řetězců je `LLMChain`, který spojuje:
- PromptTemplate (vstupní šablonu)
- LLM model
- OutputParser (např. pro formátování odpovědi)

```python
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

template = "Co je {technologie}?"
prompt = PromptTemplate(template=template, input_variables=["technologie"])
llm = OpenAI(model_name="text-davinci-003")
chain = LLMChain(prompt=prompt, llm=llm)

response = chain.run(technologie="AI")
print(response)
```

---

### Paměť (Memory types)

Paměť (Memory) umožňuje udržovat kontext mezi jednotlivými dotazy. Například v chatbotu, kde je důležité si pamatovat předchozí konverzaci.

V LangChain existují různé typy paměti:
- `ConversationBufferMemory` – ukládá celý historický dialog
- `ConversationSummaryMemory` – shrnuje předchozí konverzaci do jednoho výstupu

```python
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
```

Použití paměti:
```python
chain = LLMChain(prompt=prompt, llm=llm, memory=memory)
```

---

### Nahrávače dokumentů (Document loaders variety)

LangChain podporuje mnoho typů nahrávačů dokumentů – od textových souborů až po PDF nebo webových stránek.

Příklady:
- `TextLoader` – načte jednoduchý text
- `PDFLoader` – načte obsah PDF souboru
- `WebBaseLoader` – načte data z webové stránky

```python
from langchain.document_loaders import TextLoader

loader = TextLoader("data.txt")
documents = loader.load()
```

---

### Rozdělovače textu (Text splitters strategies)

Při práci s velkými dokumenty je důležité je rozdělit na menší části, aby se vešly do vstupu LLM.

Typy rozdělovačů:
- `RecursiveCharacterSplitter` – rozdělí podle znaků a odstavců
- `TokenTextSplitter` – rozdělí podle tokenů (přesnější pro modely)

```python
from langchain.text_splitter import RecursiveCharacterSplitter

splitter = RecursiveCharacterSplitter(chunk_size=1000, chunk_overlap=20)
chunks = splitter.split_documents(documents)
```

---

### Výstupní parser (Output parsers)

Parser převádí výstup z LLM do požadovaného formátu – například JSON nebo seznam.

Příklady:
- `JsonOutputParser` – převede odpověď na JSON
- `ListOutputParser` – rozdělí výstup na seznam

```python
from langchain.output_parsers import JsonOutputParser

parser = JsonOutputParser()
```

---

### LLM obálky (LLM wrappers)

LLM wrapper je nástroj, který přidává rozhraní pro komunikaci s různými LLM modely. Například `OpenAI`, `HuggingFaceHub`, nebo `Anthropic`.

```python
from langchain_openai import OpenAI

llm = OpenAI(model_name="text-davinci-003")
```

Wrapper je klíčový, protože umožňuje jednotnou práci s různými modely – např. přepnutí z OpenAI na Hugging Face bez změny kódu.

---

## 3. Důležité detaily

### Časté chyby a tipy na jejich vyhnutí

- **Nepředávání paměti mezi dotazy** – může vést ke ztrátě kontextu.
- **Použití příliš velkého chunku** – LLM nemusí být schopen zpracovat příliš dlouhý text.
- **Chybná konfigurace promptů** – nedostatečně přesné instrukce mohou vést k nepřesným odpovědím.

### Best practices

- Vždy používej `PromptTemplate` pro organizaci a správu vstupu.
- Testuj různé rozdělovače textu podle typu dokumentu.
- Při komunikaci s LLM využívej `OutputParser`, aby byl výstup předvídatelný.

### Performance tipy

- Optimalizuj velikost chunků – pro efektivní zpracování používej 1000–2000 tokenů.
- Používej `Cache` – např. `langchain.cache` pro opakované volání stejných dotazů.
- Využívej `Async` verze při práci s více dotazy najednou.

---

## 4. Propojení s předchozími kapitolami

Tato kapitola rozšiřuje znalosti z předchozích kapitol o jazykových modelech a jejich využití. Naučíš se, jak efektivně **spojovat jednotlivé komponenty do celkového řetězce** – tedy jak vyvíjet komplexní aplikace z jednoduchých bloků.

Zároveň vytváříme základ pro budoucí kapitoly o **řešeních v oblasti RAG (Retrieval-Augmented Generation)** a **automatizaci AI workflow**. Všechny tyto koncepty jsou klíčem k tomu, jak přejít od jednoduchého dotazu k profesionálnímu systému, který pracuje s velkými datovými objemy a komplexními logikami.

---

**Shrnutí:**  
LangChain je nástroj, který tě naučí přemýšlet o AI aplikacích jako o **řetězcích komponent**, nikoliv jako o samostatných modelech. Tato kapitola je základním krokem k tvorbě sofistikovaných chatbotů, informačních systémů a automatizovaných řešení – vše v rámci Pythonu a jazykových modelů.

Pokud jsi již zkušený s LLM, pak tento kurz ti umožní přejít od jednoduchého volání modelu k **komplexnímu a efektivnímu řešení**.


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


### Příklad 1

### 📌 Co demonstruje:
Základní instalace a první import knihovny `langchain`.

### Příklad 2

### ✅ Kompletní kód:

```python
# Instalace (pouze pokud ještě není nainstalováno)
# pip install langchain

# Importy
from langchain.llms import OpenAI  #

### Příklad 3

Příklad LLM wrapperu
import os

# Nastavení API klíče (např. z proměnné prostředí)
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Vytvoření jednoduchého LLM objektu
llm = OpenAI(model_name="text-davinci-003")

# Testování
print("LangChain je nainstalovaný a lze použít!")
```

### Příklad 4

### 🧾 Očekávaný výstup:
```
LangChain je nainstalovaný a lze použít!
```

### Příklad 5

### 🔍 Vysvětlení:
Tento příklad ukazuje, že je knihovna `langchain` správně nainstalovaná a připravená k použití. Případně by mohlo dojít k chybě, pokud není nastavený klíč API.

---

## 🧠 2. Název: Základní řetězec (Chain) s promptem

### Příklad 6

### 📌 Co demonstruje:
Vytvoření jednoduchého řetězce `LLMChain`, který kombinuje **prompt** a **LLM**.

### Příklad 7

### ✅ Kompletní kód:

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Nastavení LLM
llm = OpenAI(model_name="text-davinci-003")

# Vytvoření prompt šablony
prompt = PromptTemplate(
    input_variables=["otazka"],
    template="Odpověz na otázku: {otazka}"
)

# Vytvoření řetězce (Chain)
chain = LLMChain(llm=llm, prompt=prompt)

# Spuštění
otazka = "Jak se jmenuje hlavní město České republiky?"
vysledek = chain.run(otazka)
print(vysledek)

### Příklad 8

### 🧾 Očekávaný výstup:
```
Hlavní město České republiky je Praha.
```

### Příklad 9

### 🔍 Vysvětlení:
Používáme `PromptTemplate` pro definici vstupu a `LLMChain`, který spojuje prompt a LLM. Kód přijímá otázku a odpovídá pomocí LLM.

---

## 🧠 3. Název: Paměť (Memory) v řetězci

### Příklad 10

### 📌 Co demonstruje:
Použití paměti (`ConversationBufferMemory`) pro uchování historie konverzace.

### Příklad 11

### ✅ Kompletní kód:

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Nastavení LLM a paměti
llm = OpenAI(model_name="text-davinci-003")
memory = ConversationBufferMemory()

# Vytvoření konverzačního řetězce
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory
)

# První dotaz
response1 = conversation_chain.run("Jak se jmenuje hlavní město ČR?")
print(response1)

# Druhý dotaz (použije paměť)
response2 = conversation_chain.run("A jaké je to město?")
print(response2)

### Příklad 12

### 🧾 Očekávaný výstup:
```
Hlavní město České republiky je Praha.
Praha je hlavním městem České republiky.
```

### Příklad 13

### 🔍 Vysvětlení:
Používáme `ConversationBufferMemory`, který si pamatuje předchozí odpovědi a používá je jako kontext pro další dotazy.

---

## 🧠 4. Název: Načítání dokumentů (Document loaders)

### Příklad 14

### 📌 Co demonstruje:
Načtení jednoduchého textu pomocí `TextLoader`.

### Příklad 15

### ✅ Kompletní kód:

In [ ]:
from langchain.document_loaders import TextLoader

# Vytvoření jednoduchého souboru (pouze pro test)
with open("test.txt", "w") as f:
    f.write("LangChain je nástroj pro tvorbu aplikací s LLM.")

# Načtení dokumentu
loader = TextLoader("test.txt")
documents = loader.load()

# Výpis obsahu
for doc in documents:
    print(doc.page_content)

### Příklad 16

### 🧾 Očekávaný výstup:
```
LangChain je nástroj pro tvorbu aplikací s LLM.
```

### Příklad 17

### 🔍 Vysvětlení:
Používáme `TextLoader`, který načte obsah textového souboru. Tento dokument může být později použit v řetězcích nebo jiných komponentách.

---

## 🧠 5. Název: Rozdělení textu (Text Splitter) a parsování

### Příklad 18

### 📌 Co demonstruje:
Rozdělení delšího textu na části a vytvoření jednoduchého `OutputParser`.

### Příklad 19

### ✅ Kompletní kód:

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.output_parsers import CommaSeparatedListOutputParser

# Delší text
text = "LangChain je nástroj pro tvorbu aplikací s LLM. Využívá se pro řetězce, paměť a načítání dat."

# Rozdělení textu podle znaků
splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=0)
chunks = splitter.split_text(text)

print("Rozdělené části:")
for i, chunk in enumerate(chunks):
    print(f"{i+1}: {chunk}")

# Vytvoření jednoduchého parseru
parser = CommaSeparatedListOutputParser()
output = parser.parse("Jedna, dvě, tři")
print("Parsovaný výstup:", output)

### Příklad 20

### 🧾 Očekávaný výstup:
```
Rozdělené části:
1: LangChain je nástroj
2: pro tvorbu aplikací
3: s LLM. Využívá se
4: pro řetězce, paměť a
5: načítání dat.

Parsovaný výstup: ['Jedna', 'dvě', 'tři']
```

### Příklad 21

### 🔍 Vysvětlení:
`CharacterTextSplitter` rozděluje text podle počtu znaků. `CommaSeparatedListOutputParser` pak převádí řetězec s čárkami na seznam.

---

## 📌 Shrnutí

|

### Příklad 22

Příklad | Popis |
|--------|-------|
| 1. Instalace | Základní nastavení a importy |
| 2. Chain + Prompt | Jednoduchý řetězec s promptem |
| 3. Memory | Použití paměti v konverzaci |
| 4. Document Loaders | Načítání dokumentů z textu |
| 5. Text Splitter + Output Parser | Rozdělení textu a parsování odpovědi |

---

Pokud chceš, mohu přidat i pokračování s napojením na reálné API nebo další komponenty jako `VectorStore`, `Retriever`, atd.

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# 5 praktických cvičení pro kapitolu **LangChain základy**

---

## 1. **Název úkolu**: Vytvoření jednoduchého LangChain pipeline s LLM a promptem

### **Detailní zadání**
Vytvořte jednoduchý pipeline pomocí LangChain, který přijímá uživatelský vstup (otázku), použije model (např. `gpt-3.5-turbo`), a vrátí odpověď. Přidejte prompt, který určuje kontext pro model.

### **Vstupní data/požadavky**
- Použijte LLM model (např. `OpenAI`)
- Vstup: "Jak se dělá pizza?"
- Použijte `PromptTemplate`, který přidá kontext jako „Odpověz jasně a jednoduše.“

### **Očekávaný výstup**
Výstup bude text s odpovědí na otázku, např.:  
> "Pizza se dělá tak, že se připraví těsto, na něj se nanese omáčka a sýr, a poté se peče v troubě přibližně 12-15 minut."

### **Hints/Nápověda**
1. Použijte `ChatOpenAI` z `langchain.chat_models`.
2. Definujte `PromptTemplate` a předejte ho do `LLMChain`.
3. Vytvořte pipeline pomocí `SequentialChain` nebo pouze `LLMChain`.
4. Spusťte chain s vstupem pomocí `.run()` metody.

### **Kostra řešení**
```python
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(model_name="gpt-3.5-turbo")
template = "Odpověz jasně a jednoduše: {question}"
prompt = PromptTemplate(template=template, input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run(question="Jak se dělá pizza?")
print(response)
```

### **Bonusové rozšíření**
Přidejte logování výstupu pomocí `StreamingStdOutCallbackHandler` a přidejte podporu pro více otázek.

---

## 2. **Název úkolu**: Použití Memory v LangChain pipeline

### **Detailní zadání**
Vytvořte pipeline, který uchovává historii konverzace mezi uživatelem a LLM pomocí `ConversationBufferMemory`. Pipeline musí přijímat více otázek a odpovědí v kontextu.

### **Vstupní data/požadavky**
- Přijímání více zpráv (např. "Jak se dělá pizza?", "Kolik času to trvá?").
- Použijte `ConversationBufferMemory` a `ConversationChain`.

### **Očekávaný výstup**
Výstupy by měly obsahovat kontext z předchozích odpovědí.

### **Hints/Nápověda**
1. Vytvořte `ConversationBufferMemory`.
2. Použijte `ConversationChain` a předejte mu `llm` i `memory`.
3. Spusťte pipeline postupně s více dotazy.
4. Zkontrolujte, zda se odpovědi vztahují k minulým otázkám.

### **Kostra řešení**
```python
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo")
memory = ConversationBufferMemory()
chain = ConversationChain(llm=llm, memory=memory)

print(chain.run("Jak se dělá pizza?"))
print(chain.run("Kolik času to trvá?"))
```

### **Bonusové rozšíření**
Přidejte `ConversationSummaryMemory` pro shrnutí historie konverzace.

---

## 3. **Název úkolu**: Vytvoření RAG (Retrieval-Augmented Generation) řetězce

### **Detailní zadání**
Vytvořte jednoduchý RAG pipeline, který vyhledává informace z dokumentů a pak je použije při generování odpovědi na otázku. Použijte `VectorStore` (např. `FAISS`) a `RetrievalQA`.

### **Vstupní data/požadavky**
- Vytvořte seznam dokumentů (např. texty o technologiích).
- Vložte dokumenty do `FAISS`.
- Použijte `RetrievalQA` pro dotazování.

### **Očekávaný výstup**
Odpověď, která se odvolává na obsah dokumentu.

### **Hints/Nápověda**
1. Použijte `Document` a vytvořte seznam dokumentů.
2. Vložte je do `FAISS` pomocí `VectorStore`.
3. Předějte `vectorstore` do `RetrievalQA.from_chain_type()`.
4. Spusťte dotaz pomocí `.run()`.

### **Kostra řešení**
```python
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

documents = [Document(page_content="Technologie AI se rozvíjí rychle.")]  # příklad
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectorstore.as_retriever())
answer = qa.run("Co je AI?")
print(answer)
```

### **Bonusové rozšíření**
Přidejte `VectorStore` z reálného souboru (např. PDF) a použijte `RecursiveCharacterTextSplitter`.

---

## 4. **Název úkolu**: Vytvoření LangChain agenta pro spouštění více kroků

### **Detailní zadání**
Vytvořte jednoduchého agenta, který může provést několik kroků – např. získat informace z API nebo použít `Tool` pro řešení otázky.

### **Vstupní data/požadavky**
- Vytvořte jednoduchý `Tool`, který vrací konkrétní informace.
- Použijte `AgentExecutor` s `ZeroShotAgent`.
- Dotaz: „Kolik je 2 + 2?“ (použijte nástroj pro výpočet).

### **Očekávaný výstup**
Agent zpracuje dotaz a provede krok podle instrukce.

### **Hints/Nápověda**
1. Definujte nástroje (`tools`) pomocí `Tool` objektu.
2. Použijte `ZeroShotAgent` nebo `ConversationalAgent`.
3. Vytvořte `AgentExecutor` a spusťte dotaz.
4. Zkontrolujte, že se nástroj správně používá.

### **Kostra řešení**
```python
from langchain.agents import AgentExecutor, ZeroShotAgent
from langchain.tools import Tool

def add_numbers(x):
    return str(2 + 2)

tools = [
    Tool(name="Add", func=add_numbers, description="Použij pro sčítání")
]

prompt = ZeroShotAgent.create_prompt(tools)
agent = ZeroShotAgent(llm=OpenAI(), tools=tools)
executor = AgentExecutor(agent=agent, tools=tools)
result = executor.run("Kolik je 2 + 2?")
print(result)
```

### **Bonusové rozšíření**
Použijte `SQLDatabaseTool` nebo externí API nástroj.

---

## 5. **Název úkolu**: Vytvoření dynamického LangChain pipeline s custom promptem a výstupním formátem

###


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: *LangChain základy*

## 1. Články a tutoriály

1. **[LangChain Documentation - Quickstart](https://python.langchain.com/docs/get_started/quickstart)**  
   Oficiální rychlý start do LangChain. Vhodný pro začátečníky, kterým chceš rychle pochopit základní koncepty a zprovoznit první aplikaci.

2. **[Building a Chatbot with LangChain and OpenAI](https://towardsdatascience.com/building-a-chatbot-with-langchain-and-openai-3a1e7c4b3e9f)**  
   Praktický článek ukazující, jak vytvořit jednoduchého chatbota s využitím LangChain a OpenAI API. Obsahuje kód a vysvětlení jednotlivých komponent.

3. **[LangChain for LLM Applications - Complete Guide](https://blog.langchain.dev/langchain-v0-1-0/)**  
   Komplexní průvodce použitím LangChain pro tvorbu aplikací s LLM. Vhodné pro pokročilé uživatele, kteří chtějí přejít nad základy.

4. **[LangChain Agents Tutorial - Creating an AI Research Assistant](https://www.pinecone.io/learn/langchain-agents/)**  
   Návod na tvorbu agenta, který dokáže vyhledávat informace, provádět výpočty a komunikovat s uživatelem. Vhodný pro pochopení konceptu LangChain Agentů.

5. **[LangChain Memory and Callbacks Explained](https://blog.langchain.dev/langchain-memory-and-callbacks/)**  
   Článek se zaměřením na paměť a callbacky v LangChain, což je důležitý aspekt při tvorbě rozšířených aplikací.

---

## 2. YouTube videa

1. **[LangChain Crash Course - Build an AI Chatbot in Python](https://www.youtube.com/watch?v=Og7049QF6Z8)**  
   Délka: 35 minut  
   Tutoriál pro začátečníky, kde se naučíš vytvářet chatbot s využitím LangChain a OpenAI API. Vhodné pro ty, co preferují video výuku.

2. **[LangChain Tutorial - From Zero to Hero](https://www.youtube.com/watch?v=VJ071jY9Dgk)**  
   Délka: 58 minut  
   Komplexní přehled LangChain od základů po pokročilé funkce, včetně memory, agents a chainingu.

3. **[LangChain Agents Explained - Build Your Own AI Agent](https://www.youtube.com/watch?v=H1Z7dF60GxU)**  
   Délka: 42 minut  
   Vysvětlení, jak fungují LangChain agenti a jak je vytvořit od nuly. Doporučeno pro uživatele zájemců o autonómni systémy.

---

## 3. Knihy a oficiální dokumentace

1. **[LangChain Official Documentation](https://python.langchain.com/docs/)**  
   Oficiální dokumentace – kompletní příručka pro práci s LangChain. Obsahuje příklady, tutoriály a detailní popis funkcí.

2. **[LangChain: Building LLM Applications](https://www.amazon.com/LangChain-Building-LLM-Applications-Machine-ebook/dp/B0BZ1Z8X7S)**  
   Knihy o LangChain, která podrobně popisuje tvorbu aplikací pomocí LangChain. Doporučeno pro uživatele, kteří chtějí hluboké porozumění systému.

3. **[Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/97


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ LangChain installation</li>
<li>✓ Core concepts: Chains, Prompts</li>
<li>✓ Memory types</li>
<li>✓ Document loaders variety</li>
<li>✓ Text splitters strategies</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 34 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 13:20:33*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

